In [1]:
import pandas as pd


In [18]:
f = open("amazon/Home_and_Kitchen.json", "r")
f2 = open("amazon/Home_and_Kitchen_10.json", "w")

num_lines_before = 0
num_lines_after = 0
for line in f:
    num_lines_before += 1
    if hash(line) % 10 == 0:
        f2.write(line)
        num_lines_after += 1

f.close()
f2.close()

print(num_lines_before)
print(num_lines_after)

21928568
2190814


In [19]:
f = open("amazon/meta_Home_and_Kitchen.json", "r")

print(f.readline())

f.close()

{"category": ["Home & Kitchen", "Kitchen & Dining", "Dining & Entertaining", "Dinnerware", "Plates", "Dinner Plates"], "tech1": "", "description": ["It was a time honored tradition among the early American families that when someone deserved special praise or attention they were served dinner on the Red Plate. Observe the special occasions of each"], "fit": "", "title": "You Are Special Today Red Plate [With Red Pen]", "also_buy": ["B0001XR2F2", "B01LY51HUN", "B07CXZ9C5B", "0310258952"], "image": [], "tech2": "", "brand": "Waechtersbach USA", "feature": [], "rank": [">#39,665 in Kitchen & Dining (See Top 100 in Kitchen & Dining)", ">#93 in Kitchen & Dining > Tabletop > Dinnerware > Plates > Dinner Plates", ">#19,436 in Home & Kitchen > Kitchen & Dining > Food Service Equipment & Supplies"], "also_view": ["B0001XR2F2", "B00MOFKX1A", "B07G3LN13B", "B07CYXMFFH", "B01LY51HUN", "B06XCK743L", "B00PV4FHZI", "B07C1WTCL4", "B001ARPMD2", "B00847B0IC", "B000NCW2RE", "B06ZZ1GLQR"], "main_cat": "Am

In [29]:
import json

f = open("amazon/Home_and_Kitchen_10.json", "r")
f2 = open("amazon/Home_and_Kitchen_10_new.txt", "w")

# line = f.readline()
# line = json.loads(line)

# print(type(line['reviewText']))
num_lines = 0

for line in f:
    tmp = json.loads(line)
    try:
        if len(tmp['reviewText']) > 5:
            f2.write(tmp['reviewText'] + "\n")
            num_lines += 1
    except:
        continue

f.close()
f2.close()

print(num_lines)



2161603


In [30]:
from gensim.utils import tokenize

with open("amazon/Home_and_Kitchen_10_new.txt", "r") as f:
    tokenized_texts = [list(tokenize(text)) for text in f]
    
print(tokenized_texts[0][:10])

['I', 'don', 't', 'use', 'these', 'for', 'their', 'original', 'use', 'and']


In [ ]:
from gensim.parsing.porter import PorterStemmer

stemmer = PorterStemmer()
stemmed_texts = [[stemmer.stem(word) for word in text] for text in tokenized_texts]
stemmed_texts[0][:10]